# 판교 AI Challenge
> 참치김치찌개팀<br>
> 팀장 손찬영, 팀원 김민정 김하림 이두현 차현수
* 과제명 : [아동 및 교통약자 보호를 위한 어린이 도로보행 위험행동 분류 과제]
* 과제 링크 : https://www.aiconnect.kr/main/competition/privateDetail/200

----------------------------------------------------------------

## Package

In [16]:
import glob
import math
import os
import random
import sys
import time
import timeit
import warnings

import easydict

warnings.filterwarnings("ignore")

import gc

# Others
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Customized Source Python Files
import source.dataset as dataset

# Pytorch
import torch
import torchvision.transforms as transforms
import wandb

# from source.imbalancedDatasetSampler import ImbalancedDatasetSampler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from source.model import C3D_model, R2Plus1D_model, R3D_model
from source.model.utils.vit import TimeSformer
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from tqdm import tqdm

----------------------------------------------------------------

## Parameter Settings

In [22]:
args = easydict.EasyDict(
    {
        "experiment": "exp2 TimeSformer",  # 매번 바꿔준다.
        "wandb": False,
        "randomseed": 100,
        "dataset": "kids",  # Options: hmdb51 or ucf101 or `kids`
        "dataset_root_dir": "./dataset",
        "project_dir": os.getcwd(),
        "pretrained_model": "./pretrained/TimeSformer_divST_96x4_224_K600.pyth",  # ./pretrained/c3d-pretrained.pth, ./pretrained/TimeSformer_divST_96x4_224_K600.pyth, 'efficientnet_b4'
        "snapshot": 20,  # Store a model every snapshot epochs
        "clip_len": 5, # 74 slice.
        "num_workers": 8,
        "model": "TimeSformer",  # Options: C3D, R2Plus1D, R3D, TimeSformer, Efficientnet_LSTM
        "attention_type": "divided_space_time",
        "num_frames": 16, # model
        "img_size": 224,
        "count": 20,  # how many repeat for wandb
        "epochs": 50,
        "learning_rate": 0.005,
        "optimizer": "adam",
        "loss_function": "cross_entropy",
        "schedular": "step",
        "batch_size": 40,  # Depends on VRAM
        "multi_gpu": True,
        "autoaugment": False, # options : imagenet, svhn, cifar, False
        "smote": False,
    }
)
NAME_ELEMENTS = [args.model, time.strftime("%m%d_%H%M", time.localtime(time.time()))]
MODEL_NAME = "_".join(NAME_ELEMENTS)

In [1]:
train_size = int(len(dataset)*0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

NameError: name 'dataset' is not defined

In [23]:
print("Optimizer: {}\nLoss Function: {}\nSchedular: {}\nBatch Size: {}\n".format(args.optimizer, args.loss_function, args.schedular, args.batch_size))

Optimizer: adam
Loss Function: cross_entropy
Schedular: step
Batch Size: 40



----------------------------------------------------------------

## Randomseed & W and B

### 🚀 Setup

Start out by installing the experiment tracking library and setting up your free W&B account:<br>
`.login()` so you can log metrics to your projects<br>
If you've never used Weights & Biases before,
the call to `login` will give you a link to sign up for an account.
W&B is free to use for personal and academic projects!<br>
### 👈 Pick a `method`
The first thing we need to define is the `method`
for choosing new parameter values.

We provide the following search `methods`:
*   **`grid` Search** – Iterate over every combination of hyperparameter values.
Very effective, but can be computationally costly.
*   **`random` Search** – Select each new combination at random according to provided `distribution`s. Surprisingly effective!
*   **`bayesian` Search** – Create a probabilistic model of metric score as a function of the hyperparameters, and choose parameters with high probability of improving the metric. Works well for small numbers of continuous parameters but scales poorly.

We'll stick with `random`.

In [24]:
if args.randomseed:
    torch.manual_seed(args.randomseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(args.randomseed)
    random.seed(args.randomseed)

In [25]:
if args.wandb:
    #%%wandb
    #%env WANDB_SILENT=True
    #%env "WANDB_NOTEBOOK_NAME" "main.ipynb"
    wandb.login()
    sweep_config = {
        "name": args.experiment,
        "method": "bayes",  # grid, bayesian, random
        "metric": {
            "name": "Weighted F1 Scrore",
            "goal": "maximize",
            # 'target': 'goal value for the metric you're optimizing, for example : 0.01'
        },
        "parameters": {
            "epochs": {"values": [10, 15, 20]},
            "learning_rate": {"distribution": "uniform", "min": 1e-4, "max": 1e-3,},
            "optimizer": {"values": ["adam", "sgd", "adamw", "adadelta", "nadam"]},
            "loss_function": {"values": ["focal", "cross_entropy", "label_smooth"]},
            "schedular": {
                "values": ["step", "onecycle", "cosineannealingwarmrestarts", "swa"]
            },
            "batch_size": {"values": [20, 35]},
        },
    }
    sweep_id = wandb.sweep(sweep_config, project="tunakimchi")
    os.environ["WANDB_NOTEBOOK_NAME"] = "main.ipynb"

----------------------------------------------------------------

## Model

In [26]:
# Use GPU if available else revert to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

if args.dataset == "hmdb51":
    num_classes = 51
elif args.dataset == "ucf101":
    num_classes = 101
elif args.dataset == "kids":
    num_classes = 9
else:
    print("We only implemented hmdb and ucf datasets.")
    raise NotImplementedError

saveName = args.model + "-" + args.dataset

Device being used: cuda


In [27]:
if args.model == "C3D":
    model = C3D_model.C3D(
        model_dir=args.pretrained_model, num_classes=num_classes, pretrained=True
    )

elif args.model == "R2Plus1D":
    model = R2Plus1D_model.R2Plus1DClassifier(
        num_classes=num_classes, layer_sizes=(2, 2, 2, 2)
    )
elif args.model == "R3D":
    model = R3D_model.R3DClassifier(num_classes=num_classes, layer_sizes=(2, 2, 2, 2))

elif args.model == "TimeSformer":
    model = TimeSformer(
        img_size=args.img_size,
        num_classes=num_classes,
        num_frames=args.num_frames,
        attention_type=args.attention_type,
        pretrained_model=args.pretrained_model,
    )

elif args.model == "Efficientnet_LSTM":
    model = Efficientnet_LSTM.net(pretrain_model=args.pretrained_model, embed_size=1280, LSTM_UNITS=64, DO=0.3)

else:
    raise NotImplementedError

In [28]:
print("Training {} from scratch...".format(args.model))
print("Total params: %.2fM" % (sum(p.numel() for p in model.parameters()) / 1000000.0))
if args.multi_gpu:
    model = nn.DataParallel(model)
model.to(device)
for i, c in enumerate(model.children()):
    if i == 0:
        for k, param in enumerate(c.parameters()):
            if k <= 245:
                param.requires_grad = False
            else:
                param.requires_grad = True
        print(k)

    print('#'*25)
    print(i)
    print(list(c.parameters()))
    print('#'*25)

Training TimeSformer from scratch...
Total params: 121.27M
248
#########################
0
[Parameter containing:
tensor([[[ 1.3503e-02,  2.3845e-02, -2.0858e-01,  8.7668e-03,  2.9668e-01,
           3.5680e-02, -3.3000e-02,  2.9659e-02,  2.1445e-02, -1.9272e-01,
          -2.3144e-02, -1.4303e-02,  2.6310e-03, -1.0293e-02, -2.3999e-02,
          -1.6945e-02,  2.9305e-03,  5.2897e-02,  4.0737e-02,  8.9609e-03,
          -5.5137e-02,  9.0171e-03,  4.5456e-03, -2.5922e-02,  1.0159e-02,
           4.9536e-02,  3.0321e-03,  5.2325e-03,  3.4678e-02, -2.6220e-02,
           2.5081e-04,  2.4663e-02, -3.2667e-02, -4.9615e-03,  8.9782e-03,
           3.9877e-03,  5.0979e-02, -8.0436e-03,  1.3856e-02, -1.1134e-02,
           2.4856e-02,  2.7569e-02,  6.2446e-03,  8.5140e-03,  2.9116e-02,
           3.5907e-01, -6.8465e-03, -9.8712e-03,  1.8814e-02, -1.6475e-02,
           5.2065e-03,  2.3289e-02, -1.5290e-02, -1.3400e-02,  1.1634e-03,
          -3.1316e-03, -1.3387e-02, -1.2994e-02,  1.2351e-02,

----------------------------------------------------------------

## Dataset Preprocessing and Loading

* 베이스라인 학습을 위한 데이터 경로 설정

원활한 베이스라인 학습을 위하여 `dataset` 디렉토리 안에 다음과 같이 경로를 설정하여 데이터를 저장해야한다.<br>
train 데이터셋은 카테고리별로 별도의 디렉토리에 비디오를 분리하여 저장해야 하는데, arrangement.ipynb를 이용하면 바로 분리하여 저장시켜준다.

```
dataset
├── label
│   └── kids_labels
├── train
│   ├── driveway_walk
│   │   ├── train_0003.mp4
│   │   └── ...
│   ├── fall_down
│   │   ├── train_0002.mp4
│   │   └── ...
│   └── fighting
│   │   ├── train_0056.mp4
│   │   └── ...
│   ...
└── test (공개가 안되어있다)
    ├── test_0000.mp4
    ├── test_0000.mp4
    ├── test_0000.mp4
    ├── ...
```

학습 및 추론 전처리 과정에서 각각 train_processed, test_processed 디렉토리가 다음과 같이 생성된다.<br>
이는 dataset.py를 실행시키면 되나, 굳이 할 필요는 없다.<br>
전처리에서는 비디오에서 16프레임을 샘플링한 이미지 데이터를 비디오 파일명 디렉토리에 별도로 저장하는 과정이 수행된다.
```
dataset
├── label
│   └── kids_labels
├── train
│   ├── driveway_walk
│   │   ├── train_0003.mp4
│   │   └── ...
│   ├── fall_down
│   │   ├── train_0002.mp4
│   │   └── ...
│   └── fighting
│   │   ├── train_0056.mp4
│   │   └── ...
│   ...
│
├── train_processed
│   ├── train
│   │   ├── driveway_walk
│   │   │   ├── train_0003
│   │   │   │   ├── 00000.jpg
│   │   │   │   ├── 00001.jpg
│   │   │   └── ...
│   │   └── ...
│   └── val
│   │   ├── driveway_walk
│   │   │   ├── train_0004
│   │   │   │   ├── 00000.jpg
│   │   │   │   ├── 00001.jpg
│   │   │   │   └── ...
│   │   │   └── ...
│   │   └── ...
│
├── test
│   ├── test_0000.mp4
│   ├── test_0000.mp4
│   ├── test_0000.mp4
│   └── ...
│
└── test_processed
    ├── test_0000
    │   ├── 00000.jpg
    │   ├── 00001.jpg
    │   └── ...
    ├── test_0001
    │   ├── 00000.jpg
    │   ├── 00001.jpg
    │   └── ...
    ├── ...
```

In [29]:
class custom_dataset(Dataset):
    def __init__(self, xy):
        length = len(xy)
        x_temp = [0] * length
        y_temp = [0] * length
        for i in range(length):
            x_temp[i] = xy[i][0]
            y_temp[i] = xy[i][1]

        self.x = x_temp
        self.y = y_temp

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


def smote(train_ds, val_ds):
    sm = SMOTE(random_state=2)
    
    ################ TRAIN ################
    # Train dataset -> smote -> New train dataset!
    num_data = len(train_ds)
    train_x = torch.stack([train_ds[i][0] for i in range(num_data)])
    train_y = torch.stack([train_ds[i][1] for i in range(num_data)])
    train_x = train_x.view(num_data, -1)
    train_y = train_y.view(num_data, -1)

    train_x, train_y = sm.fit_resample(train_x, train_y)

    train_x = torch.tensor(train_x)
    train_y = torch.tensor(train_y)

    train_x = train_x.view(
        -1, 3, args.clip_len, train_ds[0][0].shape[2], train_ds[0][0].shape[3]
    )

    xy = [0] * train_x.shape[0]

    for i in range(train_x.shape[0]):
        xy[i] = (train_x[i], train_y[i])

    train_ds = custom_dataset(xy)
    
    del train_x
    del train_y
    gc.collect()
    
    ################ VALIDATION ################
    num_data = len(val_ds)
    
    val_x = torch.stack([val_ds[i][0] for i in range(num_data)])
    val_y = torch.stack([val_ds[i][1] for i in range(num_data)])
    val_x = val_x.view(num_data, -1)
    val_y = val_y.view(num_data, -1)

    val_x, val_y = sm.fit_resample(val_x, val_y)

    val_x = torch.tensor(val_x)
    val_y = torch.tensor(val_y)

    val_x = val_x.view(
        -1, 3, args.clip_len, val_ds[0][0].shape[2], val_ds[0][0].shape[3]
    )

    xy = [0] * val_x.shape[0]

    for i in range(val_x.shape[0]):
        xy[i] = (val_x[i], val_y[i])

    val_ds = custom_dataset(xy)
    
    # clear memory
    del xy
    del val_x
    del val_y
    gc.collect()
    
    return train_ds, val_ds

def autoaugment(train_ds, policies):
    
    ########################## AUTOAUGMENTATION ##########################
    if policies == 'cifar':
        policy = transforms.AutoAugmentPolicy.CIFAR10
    elif policies == 'imagenet':
        policy = transforms.AutoAugmentPolicy.IMAGENET
    elif policies == 'svhn':
        policy = transforms.AutoAugmentPolicy.SVHN
    transform = transforms.Compose([
        transforms.ConvertImageDtype(torch.uint8),
        transforms.AutoAugment(),
    ])
    transform2 = transforms.Compose([
        transforms.ConvertImageDtype(torch.float32),
    ])
    ######################################################################
    
    num_data = len(train_ds)
    
    train_y = torch.stack([train_ds[i][1] for i in range(num_data)])
    train_y = train_y.view(num_data, -1)
    train_y = torch.tensor(train_y)
    
    train_x = torch.stack([train_ds[i][0] for i in range(num_data)])
    # train_x = train_x.type(torch.uint8)
    transformed_img = []
    for j in range(train_x.size(0)):
        img = train_x[j, :, :, :, :]
        temp = []
        for i in range(img.size(1)):
            t_img = transform(img[:, i, :, :])
            t_im = transform2(t_img)
            temp.append(t_img)
        transformed_img.append(torch.stack(temp, dim=1))
    train_x = torch.stack(transformed_img, dim=0)
    # train_x = train_x.type(torch.float32)
    train_x = train_x.view(
        -1, 3, args.clip_len, train_ds[0][0].shape[2], train_ds[0][0].shape[3]
    )
    
    xy = [0] * train_x.shape[0]
    for i in range(train_x.shape[0]):
        xy[i] = (train_x[i], train_y[i])
    train_ds = custom_dataset(xy)
    
    del xy
    del train_x
    del train_y
    gc.collect()
    
    return train_ds

def classnum_check(loader):
    for batch_idx, (data, target) in enumerate(loader):
        print('Batch {}, classes {}, count {}'.format(
            batch_idx, *np.unique(target.numpy(), return_counts=True)))

In [30]:
print("Training model on {} dataset...".format(dataset))
train_dataset = dataset.VideoDataset(
    root_dir=args.dataset_root_dir,
    dataset=args.dataset,
    split="train",
    clip_len=args.clip_len,
    autoaug=False,
)

val_dataset = dataset.VideoDataset(
    root_dir=args.dataset_root_dir,
    dataset=args.dataset,
    split="val",
    clip_len=args.clip_len,
    autoaug=False,
)

Training model on <module 'source.dataset' from '/home/stephencha/Hub/ai-challenge/source/dataset.py'> dataset...
input_dir : ./dataset/train
output_dir : ./dataset/t2_processed_5clip_0.3ratio
resize_width : 288, resize_height : 162
crop_w : 259, crop_h : 145
Number of train videos: 2663
input_dir : ./dataset/train
output_dir : ./dataset/t2_processed_5clip_0.3ratio
resize_width : 288, resize_height : 162
crop_w : 259, crop_h : 145
Number of val videos: 670


In [31]:
if args.smote:
    train_dataset, val_dataset = smote(train_dataset, val_dataset)

In [32]:
trainval_loaders, trainval_sizes = build_dataset(args.batch_size, train_dataset, val_dataset)
# classnum_check(trainval_loaders['train'])

In [35]:
import matplotlib.pyplot as plt
plt.imshow(train_dataset[0][0].permute(1,0,2,3)[0].permute(1,2,0)

torch.Size([3, 145, 259])

In [12]:
if args.autoaugment:
    aug_train_dataset = autoaugment(train_dataset, args.autoaugment)
    print("AutoAugmentation ({} style)".format(args.autoaugment))
    Dset = [train_dataset, aug_train_dataset]
    train_dataset = torch.utils.data.ConcatDataset(Dset)
    del Dset

print(
    "Total Train set samples: {}, Val set samples: {}".format(
        len(train_dataset), len(val_dataset)
    )
)

AutoAugmentation (imagenet style)
Total Train set samples: 16200, Val set samples: 2034


----------------------------------------------------------------

## Loss Function and Optimizer

In [13]:
import timm.optim.nadam as nadam
import torchcontrib
from source.focalloss import FocalLoss
from source.label_smooth import LabelSmoothSoftmaxCEV2
from torch.optim.swa_utils import SWALR
from torchcontrib.optim import SWA

In [12]:
def build_dataset(batch_size, train_ds, val_ds):
    train_dataloader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=False, num_workers=args.num_workers,
    )
    val_dataloader = DataLoader(
        val_ds, batch_size=batch_size, num_workers=args.num_workers,
    )

    trainval_loaders = {"train": train_dataloader, "val": val_dataloader}
    trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ["train", "val"]}

    return trainval_loaders, trainval_sizes

In [15]:
def build_loss_function(lf):
    if lf == "focal":
        lf = FocalLoss()
    elif lf == "cross_entropy":
        lf = nn.CrossEntropyLoss()
    elif lf == "label_smooth":
        lf = LabelSmoothSoftmaxCEV2()
    return lf

In [16]:
def build_optimizer(model, opt, lr):
    if args.model == "C3D":
        param = [
            {"params": C3D_model.get_1x_lr_params(model), "lr": lr},
            {"params": C3D_model.get_10x_lr_params(model), "lr": lr * 10},
        ]
    elif args.model == "R2Plus1D":
        param = [
            {"params": R2Plus1D_model.get_1x_lr_params(model), "lr": lr},
            {"params": R2Plus1D_model.get_10x_lr_params(model), "lr": lr * 10},
        ]
    elif args.model == "R3D":
        param = model.parameters()
    elif args.model == "TimeSformer":
        param = model.parameters()
    elif args.model == "Efficientnet_LSTM":
        param = model.parameters()
    else:
        raise NotImplementedError

    if opt == "sgd":
        optimizer = optim.SGD(param, lr=lr, momentum=0.9, weight_decay=5e-4)
    elif opt == "adam":
        optimizer = optim.Adam(param, lr=lr, amsgrad=True)
    elif opt == "adamw":
        optimizer = optim.AdamW(param, lr=lr)
    elif opt == "adadelta":
        optimizer = optim.Adadelta(param, lr=lr)
    elif opt == "nadam":
        optimizer = nadam.Nadam(param, lr=lr)
    return optimizer

In [17]:
def build_schedular(optimizer, sche, epochs, length):
    if sche == "step":
        schedular = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    elif sche == "onecycle":
        schedular = optim.lr_scheduler.OneCycleLR(
            optimizer,
            pct_start=0.1,
            div_factor=1e5,
            max_lr=0.0001,
            epochs=epochs,
            steps_per_epoch=length,
        )
    elif sche == "cosineannealingwarmrestarts":
        schedular = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=2, eta_min=1e-5, last_epoch=-1
        )
    elif sche == "swa":
        schedular = SWALR(optimizer, swa_lr=0.01)
    return schedular

----------------------------------------------------------------

## Train
* epoch마다 학습과 검증을 실시한다.
* `./run/run_*` 디렉토리에서 저장된 가중치 파일을 확인한다.

In [18]:
# build run dir
runs = sorted(glob.glob(os.path.join(args.project_dir, "run", "run_*")))
runs.sort()

def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

size = get_dir_size(runs[-1])

if int(size) > 5000: # 디렉토리 용량이 있으면
    run_id = int(runs[-1].split("_")[-1]) + 1 if runs else 0 # 다음 번째로 저장
else: # 디렉토리 용량이 없으면 거기다 저장
    run_id = int(runs[-1].split("_")[-1])
print("run id: ", run_id)
SAVE_DIR = os.path.join(args.project_dir, "run", "run_" + str(run_id).zfill(3))
model_save_dir = os.path.join(SAVE_DIR, "models")

if int(size) > 5000: # 새로운 디렉토리를 만들어야 할때만 make model directory
    os.makedirs(model_save_dir, exist_ok=True)
    
print("save directory: ", model_save_dir)

run id:  4
save directory:  /home/stephencha/Hub/ai-challenge/run/run_004/models


In [19]:
def train():
    if args.wandb:
        with wandb.init(config=args, name=MODEL_NAME) as run:  # reinit=True
            config = wandb.config
    else:
        config = args

    trainval_loaders, trainval_sizes = build_dataset(
        config.batch_size, train_dataset, val_dataset
    )

    # standard crossentropy loss for classification
    criterion = build_loss_function(config.loss_function)
    optimizer = build_optimizer(model, opt=config.optimizer, lr=config.learning_rate)

    # the scheduler divides the lr by 10 every 10 epochs
    if config.schedular == "swa":
        optimizer = torchcontrib.optim.SWA(optimizer)

    scheduler = build_schedular(
        optimizer,
        sche=config.schedular,
        epochs=config.epochs,
        length=len(trainval_loaders["train"]),
    )

    best_score = 0  # np.Inf
    for epoch in range(config.epochs):
        # each epoch has a training and validation step
        for phase in ["train", "val"]:
            start_time = timeit.default_timer()

            # reset the running loss and corrects
            running_loss = 0.0
            running_corrects = 0.0

            # set model to train() or eval() mode depending on whether it is trained
            # or being validated. Primarily affects layers such as BatchNorm or Dropout.
            if phase == "train":
                # scheduler.step() is to be called once every epoch during training
                scheduler.step()
                model.train()
            else:
                model.eval()

            epoch_labels = []
            epoch_preds = []

            for inputs, labels in tqdm(trainval_loaders[phase]):
                # move inputs and labels to the device the training is taking place on
                inputs = Variable(inputs, requires_grad=True).to(device)
                labels = Variable(labels).to(device)
                optimizer.zero_grad()

                if phase == "train":
                    outputs = model(inputs)
                else:
                    with torch.no_grad():
                        outputs = model(inputs)

                probs = nn.Softmax(dim=1)(outputs)
                preds = torch.max(probs, 1)[1]
                loss = criterion(outputs, labels)

                if phase == "train":
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                epoch_labels.extend(labels.tolist())
                epoch_preds.extend(preds.tolist())

            epoch_loss = running_loss / trainval_sizes[phase]
            epoch_acc = running_corrects.double() / trainval_sizes[phase]

            epoch_score = f1_score(epoch_preds, epoch_labels, average="weighted")
            print(f"{phase} | EPOCH {epoch} Weighted F1 SCORE: {epoch_score}")

            print(
                "[{}] Epoch: {}/{} Loss: {} Acc: {}".format(
                    phase, epoch + 1, config.epochs, epoch_loss, epoch_acc
                )
            )
            stop_time = timeit.default_timer()
            print("Execution time: " + str(stop_time - start_time) + "\n")

            if args.wandb and phase == "val":
                wandb.log(
                    {"Weighted F1 Scrore": epoch_score,}
                )

            if epoch_score > best_score and phase == "val":
                print(
                    f"Validation Weighted F1 Score increased ({best_score:.6f} --> {epoch_score:.6f}).  Saving model ..."
                )
                model_path = os.path.join(
                    # SAVE_DIR,
                    # "models",
                    model_save_dir,
                    saveName
                    + "_epoch-"
                    + str(epoch).zfill(3)
                    + "_epoch_score-{:.6f}.pt".format(epoch_score)
                    + ".pth.tar",
                )
                torch.save(
                    {
                        "epoch": epoch + 1,
                        "state_dict": model.state_dict(),
                        "opt_dict": optimizer.state_dict(),
                    },
                    model_path,
                )
                print("Save model at {}\n".format(model_path))
                best_score = epoch_score

In [20]:
if args.wandb:
    wandb.agent(sweep_id, train, count=args.count)
    run.finish()
else:
    train()

 50%|████▉     | 202/405 [04:31<04:32,  1.34s/it]


RuntimeError: Caught RuntimeError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/usr/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/usr/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 83, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/usr/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 83, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/usr/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 55, in default_collate
    return torch.stack(batch, 0, out=out)
RuntimeError: stack expects each tensor to be equal size, but got [] at entry 0 and [1] at entr

In [ ]:
print("Done!")

----------------------------------------------------------------

## Inference
* 루트 디렉토리에 생성된 `submit.csv` 파일을 확인하고, 제출합니다.

In [ ]:
cls_li = [
    "driveway_walk",
    "fall_down",
    "fighting",
    "jay_walk",
    "normal",
    "putup_umbrella",
    "ride_cycle",
    "ride_kick",
    "ride_moto",
]

DATA_DIR = args.dataset_root_dir  # os.path.join(PROJECT_DIR, '')
batch_size = 4

In [ ]:
runs_pt = os.listdir(model_save_dir)
runs_pt.sort()
model_path = model_save_dir + "/" + runs_pt[-1]  # Latest
checkpoint = torch.load(model_path)
print(f"Initializing weights from: {model_path.split('/')[-1]}...")
model.load_state_dict(checkpoint["state_dict"])
print("Total params: %.2fM" % (sum(p.numel() for p in model.parameters()) / 1000000.0))

print("Model Inference on {} dataset...".format(dataset))

if os.path.isdir(os.path.join(DATA_DIR, "test_processed")):
    preprocess = False
else:
    preprocess = True

test_dataset = dataset.TestDataset(
    root_dir=args.dataset_root_dir, dataset=dataset, clip_len=16, preprocess=preprocess
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)

model.eval()
start_time = timeit.default_timer()

pred_li = []
for inputs in tqdm(test_dataloader):
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(inputs)

    probs = nn.Softmax(dim=1)(outputs)
    preds = torch.max(probs, 1)[1]
    pred_li.extend(preds.tolist())

stop_time = timeit.default_timer()
print("Execution time: " + str(stop_time - start_time) + "\n")

----------------------------------------------------------------

## Submission

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["class"] = [cls_li[int(pred)] for pred in pred_li]
sample_submission.to_csv("submit_{}.csv".format(model_path.split('/')[-1]), index=False)

----------------------------------------------------------------